# Vector Embeddings: From Zero to Hero with Python & LangChain

## Table of Contents
1. [Introduction to Vector Embeddings](#introduction)
2. [Understanding Vector Spaces](#vector-spaces)
3. [Types of Embeddings](#types-of-embeddings)
4. [Setting Up the Environment](#setup)
5. [Basic Embeddings with Python](#basic-embeddings)
6. [Working with LangChain Embeddings](#langchain-embeddings)
7. [Vector Stores and Similarity Search](#vector-stores)
8. [Building a RAG System](#rag-system)
9. [Advanced Techniques](#advanced-techniques)
10. [Best Practices](#best-practices)

---

## 1. Introduction to Vector Embeddings {#introduction}

Vector embeddings are numerical representations of text, images, or other data types that capture semantic meaning in a high-dimensional space. They enable machines to understand and process human language by converting words, sentences, or documents into vectors of real numbers.

### Key Concepts:
- **Semantic Similarity**: Similar concepts are placed closer together in vector space
- **Dimensionality**: Typical embeddings range from 128 to 4096 dimensions
- **Dense Representations**: Unlike sparse one-hot encodings, embeddings are dense vectors

### Why Vector Embeddings Matter:
- Enable semantic search beyond keyword matching
- Power recommendation systems
- Essential for modern NLP applications
- Foundation for RAG (Retrieval Augmented Generation) systems

---

## 2. Understanding Vector Spaces {#vector-spaces}

Vector embeddings exist in high-dimensional spaces where:
- Each dimension represents a learned feature
- Distance between vectors indicates semantic similarity
- Common distance metrics: Cosine similarity, Euclidean distance, Dot product

### Mathematical Foundation:

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine

# Example: Simple word vectors
word_vectors = {
    'king': np.array([0.2, 0.8, 0.1, 0.9]),
    'queen': np.array([0.3, 0.7, 0.2, 0.8]),
    'man': np.array([0.1, 0.9, 0.05, 0.95]),
    'woman': np.array([0.2, 0.8, 0.15, 0.85])
}

# Calculate cosine similarity
def cosine_similarity(v1, v2):
    return 1 - cosine(v1, v2)

# Example: king - man + woman ≈ queen
result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']
similarity = cosine_similarity(result, word_vectors['queen'])
print(f"Similarity between result and 'queen': {similarity:.3f}")

---

## 3. Types of Embeddings {#types-of-embeddings}

### 3.1 Word Embeddings
- **Word2Vec**: Skip-gram and CBOW models
- **GloVe**: Global Vectors for Word Representation
- **FastText**: Subword information inclusion

### 3.2 Sentence/Document Embeddings
- **Sentence-BERT**: Bidirectional sentence representations
- **Universal Sentence Encoder**: Google's sentence embedding model
- **Doc2Vec**: Document-level embeddings

### 3.3 Modern Transformer-based Embeddings
- **BERT**: Bidirectional Encoder Representations
- **RoBERTa**: Robustly Optimized BERT Pretraining
- **OpenAI Embeddings**: GPT-based embedding models

---

## 4. Setting Up the Environment {#setup}

In [ ]:
%%bash
# Install required packages
pip install langchain
pip install langchain-openai
pip install langchain-community
pip install chromadb
pip install sentence-transformers
pip install numpy
pip install pandas
pip install matplotlib
pip install seaborn
pip install scikit-learn

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any

# LangChain imports
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Set up API keys (replace with your actual keys)
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

---

## 5. Basic Embeddings with Python {#basic-embeddings}

### 5.1 Creating Simple Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample sentences
sentences = [
    "The cat sits on the mat",
    "A feline rests on a rug", 
    "Dogs love to play fetch",
    "Canines enjoy retrieving balls",
    "Machine learning is fascinating",
    "AI algorithms are interesting"
]

# Generate embeddings
embeddings = model.encode(sentences)
print(f"Embedding shape: {embeddings.shape}")
print(f"First embedding: {embeddings[0][:5]}...")  # Show first 5 dimensions

### 5.2 Visualizing Embeddings

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce dimensionality for visualization
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Create visualization
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

# Add labels
for i, sentence in enumerate(sentences):
    plt.annotate(sentence[:20] + "...", 
                (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                xytext=(5, 5), textcoords='offset points')

plt.title('2D Visualization of Sentence Embeddings')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.tight_layout()
plt.show()

### 5.3 Similarity Calculation

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate similarity matrix
similarity_matrix = cosine_similarity(embeddings)

# Create heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, 
            annot=True, 
            cmap='coolwarm', 
            xticklabels=[s[:15] + "..." for s in sentences],
            yticklabels=[s[:15] + "..." for s in sentences])
plt.title('Cosine Similarity Between Sentences')
plt.tight_layout()
plt.show()

---

## 6. Working with LangChain Embeddings {#langchain-embeddings}

### 6.1 OpenAI Embeddings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# Initialize OpenAI embeddings
openai_embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Generate embeddings for documents
documents = [
    "LangChain is a framework for developing applications powered by language models.",
    "Vector databases store and retrieve high-dimensional vectors efficiently.",
    "Retrieval Augmented Generation combines retrieval with generation for better answers.",
    "Embeddings capture semantic meaning in numerical form."
]

# Create embeddings
doc_embeddings = openai_embeddings.embed_documents(documents)
print(f"Number of documents: {len(doc_embeddings)}")
print(f"Embedding dimension: {len(doc_embeddings[0])}")

# Query embedding
query = "What is LangChain used for?"
query_embedding = openai_embeddings.embed_query(query)
print(f"Query embedding dimension: {len(query_embedding)}")

### 6.2 HuggingFace Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize HuggingFace embeddings
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Generate embeddings
hf_doc_embeddings = hf_embeddings.embed_documents(documents)
hf_query_embedding = hf_embeddings.embed_query(query)

print(f"HuggingFace embedding dimension: {len(hf_doc_embeddings[0])}")

### 6.3 Comparing Different Embedding Models

In [ ]:
def compare_embeddings(documents, query, embeddings_list, model_names):
    """Compare different embedding models"""
    results = {}
    
    for embeddings, name in zip(embeddings_list, model_names):
        # Get document and query embeddings
        doc_emb = embeddings.embed_documents(documents)
        query_emb = embeddings.embed_query(query)
        
        # Calculate similarities
        similarities = []
        for doc_embedding in doc_emb:
            similarity = cosine_similarity([query_emb], [doc_embedding])[0][0]
            similarities.append(similarity)
        
        results[name] = similarities
    
    return results

# Compare models
embedding_models = [openai_embeddings, hf_embeddings]
model_names = ["OpenAI", "HuggingFace"]

comparison_results = compare_embeddings(documents, query, embedding_models, model_names)

# Visualize comparison
df = pd.DataFrame(comparison_results, index=[f"Doc {i+1}" for i in range(len(documents))])
df.plot(kind='bar', figsize=(12, 6))
plt.title('Similarity Scores: Different Embedding Models')
plt.ylabel('Cosine Similarity')
plt.legend(title='Model')
plt.tight_layout()
plt.show()

---

## 7. Vector Stores and Similarity Search {#vector-stores}

### 7.1 ChromaDB Vector Store

In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# Sample documents
sample_texts = [
    "Vector embeddings are numerical representations of text that capture semantic meaning.",
    "LangChain provides tools for building applications with large language models.",
    "Retrieval Augmented Generation improves LLM responses by incorporating relevant context.",
    "Vector databases enable efficient storage and retrieval of high-dimensional vectors.",
    "Similarity search finds the most relevant documents based on semantic similarity.",
    "Natural Language Processing has been revolutionized by transformer models.",
    "Machine learning algorithms can learn patterns from large datasets.",
    "Deep learning uses neural networks with multiple layers to process information."
]

# Create vector store
vectorstore = Chroma.from_texts(
    texts=sample_texts,
    embedding=hf_embeddings,
    persist_directory="./chroma_db"
)

# Perform similarity search
query = "How do vector embeddings work?"
relevant_docs = vectorstore.similarity_search(query, k=3)

print("Query:", query)
print("\nMost relevant documents:")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. {doc.page_content}")

### 7.2 FAISS Vector Store

In [ ]:
from langchain.vectorstores import FAISS

# Create FAISS vector store
faiss_vectorstore = FAISS.from_texts(
    texts=sample_texts,
    embedding=hf_embeddings
)

# Similarity search with scores
docs_with_scores = faiss_vectorstore.similarity_search_with_score(query, k=3)

print("FAISS Similarity Search Results:")
for doc, score in docs_with_scores:
    print(f"Score: {score:.4f}")
    print(f"Content: {doc.page_content}\n")

### 7.3 Advanced Vector Store Operations

In [ ]:
# Add more documents
new_texts = [
    "Prompt engineering is crucial for getting good results from language models.",
    "Fine-tuning adapts pre-trained models to specific tasks or domains.",
    "Few-shot learning enables models to learn from just a few examples."
]

vectorstore.add_texts(new_texts)

# Search with metadata filtering
vectorstore_with_metadata = Chroma.from_texts(
    texts=sample_texts + new_texts,
    embedding=hf_embeddings,
    metadatas=[
        {"category": "embeddings", "difficulty": "beginner"},
        {"category": "langchain", "difficulty": "beginner"},
        {"category": "rag", "difficulty": "intermediate"},
        {"category": "vectordb", "difficulty": "intermediate"},
        {"category": "search", "difficulty": "beginner"},
        {"category": "nlp", "difficulty": "advanced"},
        {"category": "ml", "difficulty": "intermediate"},
        {"category": "dl", "difficulty": "advanced"},
        {"category": "prompt", "difficulty": "intermediate"},
        {"category": "finetune", "difficulty": "advanced"},
        {"category": "fewshot", "difficulty": "advanced"}
    ]
)

# Search with metadata filter
filtered_docs = vectorstore_with_metadata.similarity_search(
    query,
    k=3,
    filter={"difficulty": "beginner"}
)

print("Filtered search results (beginner level):")
for doc in filtered_docs:
    print(f"- {doc.page_content}")

---

## 8. Building a RAG System {#rag-system}

### 8.1 Document Processing Pipeline

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Sample long document
long_document = """
Vector embeddings have revolutionized natural language processing and information retrieval. 
These numerical representations capture semantic meaning in high-dimensional spaces, enabling 
machines to understand and process human language more effectively.

The concept of embeddings dates back to word2vec, which introduced the idea of representing 
words as dense vectors. This approach moved beyond simple one-hot encodings to capture 
relationships between words based on their context and usage patterns.

Modern embedding models like BERT, RoBERTa, and GPT have further advanced the field by 
providing contextualized representations. These models understand that the same word can 
have different meanings depending on its context, leading to more accurate and nuanced 
text understanding.

Vector databases have emerged as specialized storage solutions for embeddings, offering 
efficient similarity search capabilities. Popular vector databases include Pinecone, 
Weaviate, Chroma, and FAISS, each with unique features and optimization strategies.

Retrieval Augmented Generation (RAG) represents a significant advancement in language 
model applications. By combining retrieval mechanisms with generative models, RAG systems 
can provide more accurate, up-to-date, and contextually relevant responses.
"""

# Split document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    length_function=len
)

chunks = text_splitter.split_text(long_document)
print(f"Document split into {len(chunks)} chunks")

# Create vector store from chunks
rag_vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=hf_embeddings,
    persist_directory="./rag_chroma"
)

### 8.2 RAG Chain Implementation

In [ ]:
# Initialize language model
llm = OpenAI(temperature=0.7, openai_api_key=os.environ["OPENAI_API_KEY"])

# Create retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=rag_vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# Test the RAG system
questions = [
    "What are vector embeddings?",
    "How have embedding models evolved over time?",
    "What is Retrieval Augmented Generation?",
    "What are some popular vector databases?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    print("-" * 50)
    
    result = qa_chain({"query": question})
    print(f"Answer: {result['result']}")
    
    print("\nSource documents:")
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"{i}. {doc.page_content[:100]}...")

### 8.3 Custom RAG Implementation

In [ ]:
class CustomRAG:
    def __init__(self, vectorstore, llm, top_k=3):
        self.vectorstore = vectorstore
        self.llm = llm
        self.top_k = top_k
    
    def retrieve_context(self, query):
        """Retrieve relevant documents for the query"""
        docs = self.vectorstore.similarity_search(query, k=self.top_k)
        context = "\n\n".join([doc.page_content for doc in docs])
        return context, docs
    
    def generate_response(self, query, context):
        """Generate response using retrieved context"""
        prompt = f"""
        Context information:
        {context}
        
        Question: {query}
        
        Based on the context information provided, please answer the question. 
        If the context doesn't contain enough information to answer the question, 
        please say so.
        
        Answer:
        """
        
        response = self.llm(prompt)
        return response
    
    def query(self, question):
        """Main query method"""
        context, source_docs = self.retrieve_context(question)
        response = self.generate_response(question, context)
        
        return {
            'question': question,
            'answer': response,
            'context': context,
            'source_documents': source_docs
        }

# Initialize custom RAG
custom_rag = CustomRAG(rag_vectorstore, llm)

# Test custom RAG
test_query = "What makes modern embedding models better than older ones?"
result = custom_rag.query(test_query)

print(f"Question: {result['question']}")
print(f"Answer: {result['answer']}")
print(f"\nNumber of source documents: {len(result['source_documents'])}")

---

## 9. Advanced Techniques {#advanced-techniques}

### 9.1 Hybrid Search (Semantic + Keyword)

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Create BM25 retriever for keyword search
bm25_retriever = BM25Retriever.from_texts(chunks)
bm25_retriever.k = 3

# Create vector retriever for semantic search
vector_retriever = rag_vectorstore.as_retriever(search_kwargs={"k": 3})

# Combine both retrievers
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.3, 0.7]  # Give more weight to semantic search
)

# Test hybrid search
hybrid_query = "vector database optimization"
hybrid_docs = ensemble_retriever.get_relevant_documents(hybrid_query)

print(f"Hybrid search results for: {hybrid_query}")
for i, doc in enumerate(hybrid_docs, 1):
    print(f"{i}. {doc.page_content[:100]}...")

### 9.2 Embedding Fine-tuning

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Sample training data for fine-tuning
training_examples = [
    InputExample(texts=['Vector embeddings', 'Numerical representations of text'], label=1.0),
    InputExample(texts=['LangChain framework', 'Building LLM applications'], label=1.0),
    InputExample(texts=['RAG system', 'Retrieval Augmented Generation'], label=1.0),
    InputExample(texts=['Vector database', 'Machine learning models'], label=0.3),
    InputExample(texts=['Semantic search', 'Keyword matching'], label=0.2),
]

# Fine-tune embedding model (example - requires more data in practice)
def fine_tune_embeddings(model_name, training_examples, output_path):
    # Load pre-trained model
    model = SentenceTransformer(model_name)
    
    # Create data loader
    train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=16)
    
    # Define loss function
    train_loss = losses.CosineSimilarityLoss(model)
    
    # Fine-tune (minimal example)
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=10,
        output_path=output_path
    )
    
    return model

# Note: This is a simplified example - real fine-tuning requires more data and careful setup
print("Fine-tuning example prepared (requires substantial training data for real use)")

### 9.3 Multi-modal Embeddings

In [ ]:
# Example of handling different content types
class MultiModalEmbedder:
    def __init__(self, text_embedder, image_embedder=None):
        self.text_embedder = text_embedder
        self.image_embedder = image_embedder
    
    def embed_text(self, text):
        return self.text_embedder.embed_query(text)
    
    def embed_mixed_content(self, content_list):
        embeddings = []
        for content in content_list:
            if content['type'] == 'text':
                emb = self.embed_text(content['data'])
                embeddings.append(emb)
            # Add image handling when available
        return embeddings

# Initialize multi-modal embedder
multi_modal = MultiModalEmbedder(hf_embeddings)

# Example usage
mixed_content = [
    {'type': 'text', 'data': 'Vector embeddings in machine learning'},
    {'type': 'text', 'data': 'Deep learning for natural language processing'}
]

embeddings = multi_modal.embed_mixed_content(mixed_content)
print(f"Generated {len(embeddings)} embeddings for mixed content")

---

## 10. Best Practices {#best-practices}

### 10.1 Embedding Model Selection

In [ ]:
def evaluate_embedding_model(model, test_queries, test_documents):
    """Evaluate embedding model performance"""
    results = {}
    
    for query in test_queries:
        query_emb = model.embed_query(query)
        doc_embs = model.embed_documents(test_documents)
        
        similarities = [
            cosine_similarity([query_emb], [doc_emb])[0][0]
            for doc_emb in doc_embs
        ]
        
        # Get top-k most similar documents
        top_indices = np.argsort(similarities)[::-1][:3]
        results[query] = {
            'similarities': similarities,
            'top_docs': [test_documents[i] for i in top_indices],
            'top_scores': [similarities[i] for i in top_indices]
        }
    
    return results

# Test different models
test_queries = [
    "machine learning algorithms",
    "natural language processing",
    "vector similarity search"
]

test_docs = [
    "Machine learning uses algorithms to learn patterns from data",
    "NLP helps computers understand human language",
    "Vector search finds similar items in high-dimensional spaces",
    "Deep learning is a subset of machine learning",
    "Transformers revolutionized natural language understanding"
]

# Evaluate models
openai_results = evaluate_embedding_model(openai_embeddings, test_queries, test_docs)
hf_results = evaluate_embedding_model(hf_embeddings, test_queries, test_docs)

print("Model evaluation completed")

### 10.2 Optimization Strategies

In [ ]:
# Chunking strategies
def optimize_chunking(text, chunk_sizes, overlap_ratios):
    """Test different chunking strategies"""
    results = {}
    
    for chunk_size in chunk_sizes:
        for overlap_ratio in overlap_ratios:
            overlap = int(chunk_size * overlap_ratio)
            
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=overlap
            )
            
            chunks = splitter.split_text(text)
            
            results[f"size_{chunk_size}_overlap_{overlap_ratio}"] = {
                'num_chunks': len(chunks),
                'avg_chunk_length': np.mean([len(chunk) for chunk in chunks]),
                'chunks': chunks
            }
    
    return results

# Test chunking strategies
chunk_sizes = [100, 200, 500]
overlap_ratios = [0.1, 0.2, 0.3]

chunking_results = optimize_chunking(long_document, chunk_sizes, overlap_ratios)

print("Chunking optimization results:")
for strategy, result in chunking_results.items():
    print(f"{strategy}: {result['num_chunks']} chunks, avg length: {result['avg_chunk_length']:.1f}")

### 10.3 Performance Monitoring

In [ ]:
import time
from functools import wraps

def monitor_performance(func):
    """Decorator to monitor function performance"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        
        print(f"{func.__name__} executed in {end_time - start_time:.4f} seconds")
        return result
    return wrapper

@monitor_performance
def batch_embed_documents(embedder, documents, batch_size=10):
    """Embed documents in batches for better performance"""
    all_embeddings = []
    
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        batch_embeddings = embedder.embed_documents(batch)
        all_embeddings.extend(batch_embeddings)
    
    return all_embeddings

# Test batch processing
large_doc_set = sample_texts * 10  # Create larger dataset
batch_embeddings = batch_embed_documents(hf_embeddings, large_doc_set)
print(f"Processed {len(batch_embeddings)} embeddings in batches")

### 10.4 Quality Assurance

In [ ]:
def validate_embeddings(embeddings, expected_dim=None, check_norms=True):
    """Validate embedding quality"""
    issues = []
    
    # Check dimensions
    if expected_dim and len(embeddings[0]) != expected_dim:
        issues.append(f"Dimension mismatch: expected {expected_dim}, got {len(embeddings[0])}")
    
    # Check for NaN or infinite values
    for i, emb in enumerate(embeddings):
        if np.any(np.isnan(emb)):
            issues.append(f"NaN values found in embedding {i}")
        if np.any(np.isinf(emb)):
            issues.append(f"Infinite values found in embedding {i}")
    
    # Check norms (optional)
    if check_norms:
        norms = [np.linalg.norm(emb) for emb in embeddings]
        if any(norm < 0.1 or norm > 10 for norm in norms):
            issues.append("Unusual embedding norms detected")
    
    return issues

# Validate embeddings
validation_issues = validate_embeddings(doc_embeddings, expected_dim=1536)
if validation_issues:
    for issue in validation_issues:
        print(f"Warning: {issue}")
else:
    print("All embeddings passed validation")

---

## Conclusion

This notebook covered vector embeddings from basic concepts to advanced implementations using Python and LangChain. Key takeaways include:

1. **Understanding vector spaces** and semantic similarity
2. **Choosing appropriate embedding models** for your use case
3. **Building efficient vector stores** for similarity search
4. **Implementing RAG systems** for enhanced language model applications
5. **Optimizing performance** through proper chunking and batching
6. **Monitoring and validating** embedding quality

### Next Steps:
- Experiment with different embedding models for your specific domain
- Implement hybrid search strategies combining semantic and keyword search
- Explore fine-tuning embeddings for specialized applications
- Build production-ready RAG systems with proper error handling and monitoring

### Resources:
- [LangChain Documentation](https://python.langchain.com/)
- [Sentence Transformers](https://www.sbert.net/)
- [Vector Database Comparison](https://github.com/vector-databases/vector-databases)
- [Embedding Model Leaderboard](https://huggingface.co/spaces/mteb/leaderboard)